In [943]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [1123]:
class DataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.refract = 'c5ac5e60-7da0-429f-98e8-19ccdd77d835'
        self.refract_cluna = '5b0257c3-e93b-49d4-93f6-e370bc3b3f50'
        self.lping = '879d0260-93a9-4576-a1bd-2cc3dc26bf13'
        self.ystaking = '3ff0fc49-5a0d-4cdf-a8ab-33f8ea7e755f'
        self.ystaking_farm = '05d91866-0193-4231-b2ca-1774fbd5742a'
        self.swaps = '1bfd8019-89a1-470d-8868-60d71e57d1d0'
        self.router = '69b149fb-81ba-4860-aac1-b17f0d6d7688'
        self.collector = '2ab62a07-3882-48e6-bdc6-d9e592aee2d8'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
        self.dates_to_mark = pd.DataFrame([
            ['2022-02-13', '2022-02-13',1600000,'Prism Forge'], 
            ['2022-03-06', '2022-03-06',2100000,'Prism Farm']], 
            columns=['text_date','date','height','text']
        )
        
    def load_from_url(self):
        self.ystaking_farm_df = self.claim(self.ystaking_farm)
        self.lping_df = self.claim(self.lping)
        self.refract_df = self.claim(self.refract)
        self.refract_cluna_df = self.claim(self.refract_cluna)
        self.ystaking_df = self.claim(self.ystaking)
        self.swaps_df = self.claim(self.swaps)
        self.router_df = self.claim(self.router)
        self.collector_df = self.claim(self.collector)
    
    def write_to_csv(self):
        self.ystaking_farm_df.to_csv(f'{self.path_to_data}/ystaking_farm.csv')
        self.lping_df.to_csv(f'{self.path_to_data}/lping.csv')
        self.refract_df.to_csv(f'{self.path_to_data}/refract.csv')
        self.refract_cluna_df.to_csv(f'{self.path_to_data}/refract_cluna.csv')
        self.ystaking_df.to_csv(f'{self.path_to_data}/ystaking.csv')
        self.swaps_df.to_csv(f'{self.path_to_data}/swaps.csv')
        self.router_df.to_csv(f'{self.path_to_data}/router.csv')
        self.collector_df.to_csv(f'{self.path_to_data}/collector.csv')
        
    def load_from_csv(self):
        self.ystaking_farm_df = pd.read_csv(f'{self.path_to_data}/ystaking_farm.csv', index_col=0)
        self.lping_df = pd.read_csv(f'{self.path_to_data}/lping.csv', index_col=0)
        self.refract_df = pd.read_csv(f'{self.path_to_data}/refract.csv', index_col=0)
        self.refract_cluna_df = pd.read_csv(f'{self.path_to_data}/refract_cluna.csv', index_col=0)
        self.ystaking_df = pd.read_csv(f'{self.path_to_data}/ystaking.csv', index_col=0)
        self.swaps_df = pd.read_csv(f'{self.path_to_data}/swaps.csv', index_col=0)
        self.router_df = pd.read_csv(f'{self.path_to_data}/router.csv', index_col=0)
        self.collector_df = pd.read_csv(f'{self.path_to_data}/collector.csv', index_col=0)
        
    def polish_lping(self):
        self.lping_df['action'] = self.lping_df.apply(lambda row: row['0_action'] if row['0_action'] == 'provide_liquidity' else row['1_action'],axis=1)
        provide_ = self.lping_df[self.lping_df.action=='provide_liquidity']
        withdraw_ = self.lping_df[self.lping_df.action=='withdraw_liquidity']
        #
        provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['3_amount'],axis=1)
        provide_['yluna_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['2_amount'],axis=1)
        provide_['sender'] = provide_['from_']
        provide_['hr'] = provide_.block_timestamp.str[:-5] + '00:00.000'
        provide_['day'] = provide_.block_timestamp.str[:-9]
        provide_ = provide_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
        provide_['amount_signed'] = provide_.yluna_amount
        provide_['type'] = 'provide_lp'
        self.provide_ = provide_
        #
        withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['2_amount'],axis=1)
        withdraw_['yluna_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['1_amount'],axis=1)
        withdraw_['sender'] = withdraw_['from_']
        withdraw_['hr'] = withdraw_.block_timestamp.str[:-5] + '00:00.000'
        withdraw_['day'] = withdraw_.block_timestamp.str[:-9]
        withdraw_ = withdraw_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
        withdraw_['amount_signed'] = -withdraw_.yluna_amount
        withdraw_['type'] = 'withdraw_lp'
        self.withdraw_ = withdraw_

        
    def polish_refracting_cluna(self):
        self.refract_cluna_df['user'] = self.refract_cluna_df['from_']
        self.refract_cluna_df_pol = self.refract_cluna_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_cluna_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_cluna_df_pol['asset_given'] = 'cLUNA'
        self.refract_cluna_df_pol['asset_received'] = 'yLUNA'
        self.refract_cluna_df_pol['operation'] = 'refraction'
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_cluna_df_pol['hr'] = self.refract_cluna_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_cluna_df_pol['day'] = self.refract_cluna_df_pol.block_timestamp.str[:-9]
        self.refract_cluna_df_pol['amount_signed'] = self.refract_cluna_df_pol.amount
        
    def polish_refracting_luna(self):
        self.refract_df['user'] = self.refract_df.apply(lambda row: row.from_ if row['0_action']=='bond_split' else row.to_, axis=1)
        self.refract_df_pol = self.refract_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_df_pol['asset_given'] = self.refract_df_pol.apply(lambda row: 'LUNA' if row['action']=='bond_split' else 'yLUNA',axis=1)
        self.refract_df_pol['asset_received'] = self.refract_df_pol.apply(lambda row: 'yLUNA' if row['action']=='bond_split' else 'LUNA',axis=1)
        self.refract_df_pol['operation'] = 'refraction'
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_df_pol['hr'] = self.refract_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_df_pol['day'] = self.refract_df_pol.block_timestamp.str[:-9]
        self.refract_df_pol['amount_signed'] = self.refract_df_pol.apply(lambda row: -row.amount 
                                                        if row.action=='burn_from' else row.amount,axis=1)
        
    def polish_refracting(self):
        self.polish_refracting_cluna()
        self.polish_refracting_luna()
        self.all_refreact = self.refract_df_pol.append(self.refract_cluna_df_pol)
        daily_delta_rf = self.all_refreact.groupby('day').amount_signed.sum().reset_index()
        daily_delta_rf = daily_delta_rf.sort_values(by='day')
        daily_delta_rf['cumsum'] = daily_delta_rf.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_rf.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_rf['Type'] = 'yLuna circulating'
        self.daily_delta_rf = daily_delta_rf
        
    def polish_ystaking(self):
        self.ystaking_df['action'] = self.ystaking_df.apply(lambda row: row['0_action'] if row['0_action']=='unbond' else row['1_action'],axis=1)
        self.ystaking_df['amount'] = self.ystaking_df.apply(lambda row: row['0_amount'] if row['0_action']=='unbond' else row['1_amount'],axis=1)
        self.ystaking_df['user'] = self.ystaking_df.apply(lambda row: row['to_'] if row.action=='unbond' else row['from_'],axis=1)
        self.ystaking_df['asset_given'] = self.ystaking_df.apply(lambda row: None if row.action=='unbond' else 'yLUNA',axis=1)
        self.ystaking_df['asset_received'] = self.ystaking_df.apply(lambda row: 'yLUNA' if row.action=='unbond' else None,axis=1)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_df['day'] = self.ystaking_df.hr.apply(str).str[:-13]
        self.ystaking_df = self.ystaking_df[['block_timestamp','tx_id','hr','day','action','amount','user','asset_given','asset_received']]
        self.ystaking_df['amount_signed'] = self.ystaking_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        
    def polish_ystaking_farm(self):
        self.ystaking_farm_df['action'] = self.ystaking_farm_df.apply(lambda row: row['3_action'] if row['3_action']=='bond' else row['1_action'], axis=1)
        self.ystaking_farm_df['amount'] = self.ystaking_farm_df.apply(lambda row: float(row['0_amount']) if row['action']=='bond' else float(row['0_amount']), axis=1)
        self.ystaking_farm_df = self.ystaking_farm_df[['block_timestamp','tx_id','sender','action','amount']]
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_farm_df['hr'] = self.ystaking_farm_df.block_timestamp.str[:-5] + '00:00.000'
        self.ystaking_farm_df['day'] = self.ystaking_farm_df.block_timestamp.str[:-9]
        self.ystaking_farm_df['amount_signed'] = self.ystaking_farm_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        
        
    def polish_simple_swaps(self):
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol
    
    def polish_router(self):
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol
    
    def polish_swaps(self):
        self.polish_simple_swaps()
        self.polish_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps
    
    def get_amount_yluna(self,row):
        #Swapping Prism for yLuna
        for i in range(-1,11):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}offer_asset'] == 'cw20:terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'):
                return row[f'{prefix}offer_amount']
        #Asking for yLuna from the pool
        for i in range(-1,11):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}ask_asset'] == 'cw20:terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'):
                return -row[f'{prefix}return_amount']
        #Swapping PRISM for yLuna
        for i in range(-1,11):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                if(row[f'{prefix}from'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                    if(row[f'{prefix}action'] == 'swap'):
                        return row[f'{i}_amount']
        #Sending yLuna to the pool
        for i in range(-1,11):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'):
                if(row[f'{prefix}from'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                    if(row[f'{prefix}action'] == 'send'):
                        return row[f'{prefix}amount']
    
    def polish_collector_and_others(self):
        lp_txs = self.withdraw_[['tx_id']].append(self.provide_[['tx_id']])\
                            .append(self.yluna_swaps[['tx_id']])
        self.collector_df.block_timestamp=self.collector_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.collector_df.block_timestamp=self.collector_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.collector_df['hr'] = self.collector_df.block_timestamp.str[:-5] + '00:00.000'
        self.collector_df['day'] = self.collector_df.block_timestamp.str[:-9]
        self.collector_df = self.collector_df[self.collector_df.tx_id.isin(set(dp.collector_df.tx_id.unique()).difference(lp_txs.tx_id.unique()))]
        self.collector_df['amount_signed'] = self.collector_df.apply(self.get_amount_yluna,axis=1)/1000000
        self.collector_df['type'] = 'collector_and_other'
        
    def polish(self):
        self.polish_lping()
        self.polish_refracting()
        self.polish_ystaking()
        self.polish_ystaking_farm()
        self.polish_swaps()
        self.polish_collector_and_others()
    
    def lp_delta(self):
        self.all_lps = self.withdraw_[['day','amount_signed','tx_id','type','block_timestamp']]\
                            .append(self.provide_[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(self.yluna_swaps[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(self.collector_df[['day','amount_signed','tx_id','type','block_timestamp']])
        daily_delta_lp = self.all_lps.groupby('day').amount_signed.sum().reset_index()
        daily_delta_lp = daily_delta_lp.sort_values(by='day')
        daily_delta_lp['cumsum'] = daily_delta_lp.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_lp.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_lp['Type'] = 'yLuna LP'
        self.daily_delta_lp = daily_delta_lp
        
    def stk_farm_delta(self):
        daily_delta_stk_farm = self.ystaking_farm_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk_farm = daily_delta_stk_farm.sort_values(by='day')
        daily_delta_stk_farm['cumsum'] = daily_delta_stk_farm.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk_farm.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk_farm['Type'] = 'yLuna Farm staked'
        self.daily_delta_stk_farm = daily_delta_stk_farm
        
    def stk_delta(self):
        daily_delta_stk = self.ystaking_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk = daily_delta_stk.sort_values(by='day')
        daily_delta_stk['cumsum'] = daily_delta_stk.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk['Type'] = 'yLuna staked'
        self.daily_delta_stk = daily_delta_stk
    
    def unused_yluna(self):
        self.y_used = self.all_deltas[self.all_deltas.Type.isin(['yLuna LP','yLuna staked','yLuna Farm staked'])]
        y_tot = self.all_deltas[self.all_deltas.Type.isin(['yLuna circulating'])]
        y_unused = self.y_used.groupby('Time').Amount.sum().rename('Amount_used').reset_index().merge(y_tot, on='Time')
        y_unused['Amount_delta'] = y_unused['Amount'] - y_unused['Amount_used']
        y_unused['Amount_delta'] = y_unused['Amount_delta'].apply(lambda x: x if x>0 else 0)
        y_unused = y_unused[['Time','Amount signed','Amount_delta','Type']]
        y_unused['Type'] = 'yLuna unused'
        y_unused.columns = ['Time','Amount signed','Amount','Type']
        self.y_unused = y_unused
        
    def daily_delta(self):
        self.lp_delta()
        self.stk_farm_delta()
        self.stk_delta()
        self.all_deltas = self.daily_delta_rf.append(self.daily_delta_stk)\
                                    .append(self.daily_delta_lp)\
                                    .append(self.daily_delta_stk_farm)
        self.unused_yluna()
        
        
    def fill_date_gaps(self, dff, extra_dates=[]):
        dd = dff.Time.unique()
        dd = [*dd,*extra_dates]
        unique_dates = pd.Series(dd).rename('Time').reset_index().drop(columns='index')
        for t in dff.Type.unique():
            df = dff[dff.Type==t].merge(unique_dates, on='Time', how='right')
            df = df[df.Type.isna()]
            df['Type'] = t
            dff = dff.append(df.fillna(0))
        dff = dff.drop_duplicates()
        return dff


In [1124]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [1125]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [1126]:
class ChartProvider:
    def __init__(self):
        pass
    
    def get_line_chart(self, df, scale_, min_date=None, max_date=None, top_padding=0):
        max_date = df['Time'].max()
        df['Amount (millions)'] = round(df['Amount']/1000000,2).apply(str)+'M'
        chart = alt.Chart(df).mark_area().encode(
            x=alt.X('Time:T',scale=alt.Scale(domain=(min_date,max_date))),
            y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,df['Amount'].max()+top_padding))),
            color=alt.Color('Type:N', 
                        scale=scale_,
                        legend=alt.Legend(
                                    orient='none',
                                    padding=5,
                                    legendY=0,
                                    direction='vertical')),
            tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d'),'Type:N','Amount (millions):N']
        )
        return chart

In [1127]:
dp = DataProvider(claim,get_url,'../data')
#dp.load_from_url()
#dp.write_to_csv()
dp.load_from_csv()
dp.polish()
dp.daily_delta()

## Refract

In [1128]:
dp.refract_cluna_df.block_timestamp

0       2022-02-14 15:33:30.638
1       2022-02-14 13:36:28.244
2       2022-02-14 23:05:30.898
3       2022-02-14 13:21:41.681
4       2022-02-17 04:05:39.063
                 ...           
2211    2022-03-07 19:29:58.462
2212    2022-03-07 19:21:28.641
2213    2022-03-07 20:09:03.861
2214    2022-03-06 18:05:03.385
2215    2022-03-07 19:14:19.107
Name: block_timestamp, Length: 2216, dtype: object

In [1129]:
dp.refract_cluna_df_pol.block_timestamp

0       2022-02-14 15:33:30
1       2022-02-14 13:36:28
2       2022-02-14 23:05:30
3       2022-02-14 13:21:41
4       2022-02-17 04:05:39
               ...         
2211    2022-03-07 19:29:58
2212    2022-03-07 19:21:28
2213    2022-03-07 20:09:03
2214    2022-03-06 18:05:03
2215    2022-03-07 19:14:19
Name: block_timestamp, Length: 2216, dtype: object

In [1130]:
daily_delta_rf = dp.daily_delta_rf

In [1131]:
daily_delta_rf.head()

,Time,Amount signed,Amount,Type
0,2022-02-14,886905.699296,886905.70,yLuna circulating
1,2022-02-15,203535.451347,1090441.15,yLuna circulating
2,2022-02-16,34802.863363,1125244.01,yLuna circulating
3,2022-02-17,390199.478500,1515443.49,yLuna circulating
4,2022-02-18,18926.171760,1534369.66,yLuna circulating


In [1132]:
cp = ChartProvider()
domain = ['yLuna circulating']
range_ = ['#f8936d']
cp.get_line_chart(daily_delta_rf, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_rf.Time.min(),
               max_date = daily_delta_rf.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## yStaking

In [1133]:
def get_max_domain_date(df, time_field, n_hours):
    if((pd.Timestamp(df[time_field].max()) - 
                pd.Timestamp(df[time_field].min())).total_seconds()/3600 < n_hours):
        max_date = (pd.Timestamp(df[time_field].min()) + pd.to_timedelta(n_hours, unit='h')).strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        max_date = df[time_field].max()
    return max_date

In [1134]:
daily_delta_stk = dp.daily_delta_stk

In [1135]:
domain = ['yLuna staked']
range_ = ['#f8936d']
cp.get_line_chart(daily_delta_stk, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [1136]:
cp.get_line_chart(dp.daily_delta_lp, 
               alt.Scale(scheme='set2'),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [1137]:
dp.all_lps.groupby(['day']).amount_signed.sum()

day
2022-02-14           35.495303
2022-02-14 12:    14953.299999
2022-02-14 13:      804.826776
2022-02-14 14:      803.721496
2022-02-14 15:     1114.799588
2022-02-14 16:     2298.423708
2022-02-14 17:     4721.609678
2022-02-14 18:      839.930314
2022-02-14 19:      384.772155
2022-02-14 20:      876.123989
2022-02-14 21:     3394.559166
2022-02-14 22:     -310.333807
2022-02-14 23:      413.984827
2022-02-15       -12021.508787
2022-02-15 00:      855.428906
2022-02-15 01:     1770.274249
2022-02-15 02:     -215.066739
2022-02-15 03:     2126.716779
2022-02-15 04:      484.282719
2022-02-15 05:     1481.999305
2022-02-15 06:     -205.391842
2022-02-15 07:      382.585522
2022-02-15 08:     1725.314190
2022-02-15 09:     -486.359310
2022-02-15 10:      211.370239
2022-02-15 11:      561.580450
2022-02-15 12:     1103.255811
2022-02-15 13:      616.878022
2022-02-15 14:      419.443868
2022-02-15 15:     6273.364755
2022-02-15 16:     1676.745373
2022-02-15 17:     -177.158237
2022

In [1138]:
cp.get_line_chart(dp.daily_delta_stk_farm, 
               alt.Scale(scheme='set2'),
               min_date = dp.daily_delta_stk_farm.Time.min(),
               max_date = dp.daily_delta_stk_farm.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [1139]:
cp.get_line_chart(dp.y_unused, 
               alt.Scale(scheme='set2'),
               min_date = y_unused.Time.min(),
               max_date = y_unused.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## All deltas

In [1140]:
all_deltas = dp.y_used.append(dp.y_unused)

In [1141]:
all_deltas = dp.fill_date_gaps(all_deltas, ['2022-02-12','2022-02-13'])

In [1142]:
c1 = cp.get_line_chart(all_deltas, 
               alt.Scale(scheme='set2'),
               min_date = all_deltas.Time.min(),
               max_date = all_deltas.Time.max(),
               top_padding = 500000
        )

c2 = alt.Chart(dp.dates_to_mark).mark_rule(color='#e45756').encode(
    x=alt.X('date'+':T',axis=alt.Axis(labels=False,title=''))
)

c3 = alt.Chart(dp.dates_to_mark).mark_text(
    color='#e45756',
    angle=0
).encode(
    x=alt.X('text_date'+':T',axis=alt.Axis(labels=False,title='')),
    y='height',
    text='text'
)

(c1 + c2 + c3).properties(width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [1143]:
dp.collector_df.sort_values(by='amount_signed', ascending=False)[['tx_id','amount_signed']]

,tx_id,amount_signed
8769,74BEBF29D21CCC2FBED938F56B7A9FBF17ABDA89E07D98140176DC6A3AB8E505,1200.000000
10967,E22CF60CC5C8E8B3B8D53C8F6F92BFF27D580DC95B67916677CD7EE37357C05E,800.000000
4796,1C9AE6F113DAA0CB1289B887EA5AF07891B919D8E3E44440E22EECF4922822E8,633.064697
13366,9AD6C927BD69598E209227B9D22473A9ECCC8E9B01EEC6468AFC0F173B09C34E,601.755171
4144,0C9AFA989CF8F1C4446B3EE52C57E96D26C6D86C1D4ACF8216FCA48610F6DD19,427.253539
...,...,...
13531,2F61C922000B85A92B4172C983D7CE67E91733264A14A764542154F763C0B0B3,-269.721367
10436,C2CB8B1CFBCDF23A718B01B9DBDCBEBB41688CD2EC85642E6418C0F13D9DEA6F,-271.018696
9263,4854A2F4CCAF86BB960155F18F5F240E75E22DB82A4A0E5DFF3664DA727E361E,-274.343527
14049,3BBF0C3E541D1032CDEDBF7755519364041995854ABD968A818CF7CB07381C57,-365.779122


In [1144]:
dp.collector_df.loc[8769]

0_action                                                                           send
0_amount                                                                   1200000000.0
0_ask_asset                           cw20:terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw
0_contract_address                         terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz
0_from                                     terra1qlza4svxszm7c2j6ll2z2nd6pa55dm2ud79jzu
0_offer_amount                                                             1200000000.0
0_offer_asset                         cw20:terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz
0_receiver                                 terra1yrc0zpwhuqezfnhdgvvh7vs5svqtgyl7pu3n6c
0_return_amount                                                          166890392743.0
0_sender                                   terra1yrc0zpwhuqezfnhdgvvh7vs5svqtgyl7pu3n6c
0_spread_amount                                                           12330104077.0
0_to                            

## Query

In [ ]:
import requests
luna_apr = float(
    requests.get('https://api.terra.dev/chart/staking-return/annualized').json(
    )[-1]['value']) * 100
et_query = requests.get(
    'https://api.extraterrestrial.money/v1/api/prices').json()
yluna_price = float(et_query['prices']['yLUNA']['price'])
luna_price = float(et_query['prices']['LUNA']['price'])
yluna_apr = luna_apr * luna_price / yluna_price

In [756]:
yluna_apr

7.950117121734308

In [757]:
luna_apr

6.81135800269274

In [758]:
luna_price

85.3051177035599

In [759]:
df = pd.DataFrame(requests.get('https://api.terra.dev/chart/staking-return/annualized').json())
df['date'] = pd.to_datetime(df['datetime'], unit='ms')
df = df[df['date'] > '2021-07-05 15:00:00']

In [760]:
df.value = df.value.apply(float).apply(lambda x: x*100)

In [761]:
mars_price_chart = alt.Chart(df).mark_line(point = True).encode(
    x=alt.X('date:T'),
    y=alt.X('value:Q'),
    tooltip=[alt.Tooltip('date:T', format='%Y-%m-%d %H:%M'),'value:Q']
).properties(width=800).properties(height=300).configure_view(strokeOpacity=0).interactive()
mars_price_chart

alt.Chart(...)